In [142]:
import pandas as pd
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np
from scipy.stats import ttest_ind
import math

In [143]:
'''READS IN TEAM STATS FOR'''
team_stats1314 = pd.read_csv('./preprocessed/team_stats1314.csv', sep=',')
# team_stats1314
team_stats1415 = pd.read_csv('./preprocessed/team_stats1415.csv', sep=',')
team_stats1516 = pd.read_csv('./preprocessed/team_stats1516.csv', sep=',')
team_stats1617 = pd.read_csv('./preprocessed/team_stats1617.csv', sep=',')
team_stats1718 = pd.read_csv('./preprocessed/team_stats1718.csv', sep=',')

dfs = [team_stats1314, team_stats1415, team_stats1516, team_stats1617, team_stats1718]

for df in dfs:
    df['Team'] = df.Team.str.replace(' ', '_')
    df['Team'] = df['Team'].str[:-1]

team_stats1718


,Unnamed: 0,Rk,Team,G,MP,FG,FGA,FG%,3P,3PA,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,WIN%
0,0,3,Golden_State_Warriors,82,240.6,42.8,85.1,0.503,11.3,28.9,...,8.4,35.1,43.5,29.3,8.0,7.5,15.5,19.6,113.5,70.7
1,1,1,Houston_Rockets,82,240.9,38.7,84.2,0.460,15.3,42.3,...,9.0,34.5,43.5,21.5,8.5,4.8,13.8,19.5,112.4,79.3
2,2,8,New_Orleans_Pelicans,82,243.4,42.7,88.3,0.483,10.2,28.2,...,8.7,35.7,44.3,26.8,8.0,5.9,14.9,19.1,111.7,58.5
3,3,2,Toronto_Raptors,82,241.8,41.3,87.4,0.472,11.8,33.0,...,9.8,34.2,44.0,24.3,7.6,6.1,13.4,21.8,111.7,72.0
4,4,6,Cleveland_Cavaliers,82,240.6,40.4,84.8,0.476,12.0,32.1,...,8.5,33.7,42.1,23.4,7.1,3.8,13.7,18.6,110.9,61.0
5,5,5,Philadelphia_76ers,82,241.2,40.8,86.6,0.472,11.0,29.8,...,10.9,36.5,47.4,27.1,8.3,5.1,16.5,22.1,109.8,63.4
6,6,12,Minnesota_Timberwolves,82,241.5,41.0,86.1,0.476,8.0,22.5,...,10.3,31.6,42.0,22.7,8.4,4.2,12.5,18.2,109.5,57.3
7,7,8,Oklahoma_City_Thunder,82,241.8,39.9,88.0,0.453,10.7,30.4,...,12.5,32.6,45.1,21.3,9.1,5.0,14.0,20.2,107.9,58.5
8,8,17,Washington_Wizards,82,242.1,39.9,85.6,0.467,9.9,26.5,...,10.0,33.1,43.1,25.2,7.9,4.3,14.6,21.3,106.6,52.4
9,9,15,Milwaukee_Bucks,82,241.8,39.7,83.0,0.478,8.8,24.7,...,8.4,31.5,39.8,23.2,8.8,5.4,13.8,21.4,106.5,53.7


In [144]:
'''READS IN TEAM STATS AGAINST'''
opp_stats1314 = pd.read_csv('./preprocessed/opp_stats1314.csv', sep=',')
opp_stats1415 = pd.read_csv('./preprocessed/opp_stats1415.csv', sep=',')
opp_stats1516 = pd.read_csv('./preprocessed/opp_stats1516.csv', sep=',')
opp_stats1617 = pd.read_csv('./preprocessed/opp_stats1617.csv', sep=',')
opp_stats1718 = pd.read_csv('./preprocessed/opp_stats1718.csv', sep=',')

opp_arr = [opp_stats1314, opp_stats1415, opp_stats1516, opp_stats1617, opp_stats1718]

for df in opp_arr:
    df['Team'] = df['Team'].str[:-1]
    

In [145]:
series_in1314 = pd.read_csv('./Playoff_Matchups/Playoff_13-14.csv', sep=',')
series_in1415 = pd.read_csv('./Playoff_Matchups/Playoff_14-15.csv', sep=',')
series_in1516 = pd.read_csv('./Playoff_Matchups/Playoff_15-16.csv', sep=',')
# series1617 = pd.read_csv('./Playoff_Matchups/Playoff_16-17.csv', sep=',')

In [146]:
def make_series(tm, series, df):
    
    if tm == 'a':
        a = series['TeamA'].tolist()
    else:
        a = series['TeamB'].tolist()
        
    team = pd.DataFrame()
    
    for x in a:
        to_concat = df[df['Team'] == x]
        frames = [team, to_concat]
        team = pd.concat(frames)
    
    return team
    

In [147]:
'''THIS MAKES THE FOR AND OPPOSING TABLES FOR THE PLAYOFFS TEAM A'''
a = 'a'
team_a1314 = make_series(a, series_in1314, team_stats1314)
team_a_opp1314 = make_series(a, series_in1314, opp_stats1314)
team_a1415 = make_series(a, series_in1415, team_stats1415)
team_a_opp1415 = make_series(a, series_in1415, opp_stats1415)
team_a1516 = make_series(a, series_in1516, team_stats1516)
team_a_opp1516 = make_series(a, series_in1516, opp_stats1516)
# team_a1617 = make_series(a, series_in1617, team_stats1617)
# team_a_opp1617 = make_series(a, series_in1617, opp_stats1617)

# team_a1314

In [148]:
'''THIS MAKES THE FOR AND OPPOSING TABLES FOR THE PLAYOFFS TEAM B'''
b = 'b'
team_b1314 = make_series(b, series_in1314, team_stats1314)
team_b_opp1314 = make_series(b, series_in1314, opp_stats1314)
team_b1415 = make_series(b, series_in1415, team_stats1415)
team_b_opp1415 = make_series(b, series_in1415, opp_stats1415)
team_b1516 = make_series(b, series_in1516, team_stats1516)
team_b_opp1516 = make_series(b, series_in1516, opp_stats1516)
# team_b1617 = make_series(b, series_in1617, team_stats1617)
# team_b_opp1617 = make_series(b, series_in1617, opp_stats1617)

# team_b1314

In [149]:
'''THIS MAKES THE WINNER DATAFRAME'''
winner1314 = series_in1314['Winner']
winner1415 = series_in1415['Winner']
winner1516 = series_in1516['Winner']
# winner1617 = series_in1617['Winner']